
## KCI 사회과학 분야 영어초록 분석(2004–2024) - 과잉어휘 시각화까지

두 개의 엑셀 파일(1: 2004–2023**, 2: 2024)을 병합하고 아래 전처리를 수행.

**전처리 규칙:**
- 사용 컬럼: `논문ID`, `영어초록`, `발행년도`, `주제분야` (영어제목은 제외함)
- 언어 필터: 별도 감지 생략 (비어있는 초록만 제거)
- 길이 규칙은 별도로 적용 안 함


# 1) 데이터 로드 및 전처리

In [ ]:
import pandas as pd

# 데이터 불러오기
df1 = pd.read_excel('KCI_SS_part1.xlsx')  # 2004–2023
df2 = pd.read_excel('KCI_SS_part2.xlsx')  # 2024

# 불러온 데이터 미리보기 (앞 3줄)
print(df1.head(3))
print(df2.head(3))

In [ ]:
# 사용하고자 하는 컬럼 추출
df1 = df1[['논문ID', '발행년도', '논문영어제목', '영어초록','주제분야']]
df2 = df2[['논문ID', '발행년도', '논문영어제목', '영어초록','주제분야']]

#데이터 병합
df = pd.concat([df1, df2], ignore_index=True)
print(f"병합된 데이터프레임 크기: {df.shape}")

# 결측치 확인 - 아래는 결측치 입니다!
print(df.isnull().sum())

In [ ]:
# 공백/빈문자열 및 결측 문자열 → NA 처리
for c in ['논문영어제목', '영어초록']:
    s = df[c].astype(str).str.strip().str.lower()
    # "nan", "none", "null", "<na>", "n/a" 같은 문자열 결측 토큰 제거
    s = s.replace(r'^(nan|none|null|<na>|n/a)$', '', regex=True)
    # 빈 문자열을 NA로 바꾸기
    s = s.replace({'': pd.NA})
    df[c] = s

# 영어초록만 필수로 남기기
df = df.dropna(subset=['영어초록'])
print(f"[OK] 영어초록 결측 제거 후 크기: {df.shape}")

# 데이터 정보 확인
print(df.shape)
print(df.columns)
df.head()

### 추후 주제별 비교를 위해 '주제분야' 컬럼 결측 보정 및 계층 분해

In [ ]:
import pandas as pd
import re

# 결측/유사결측 정의
NULL_LIKE = {'', 'nan', 'none', 'null', 'na', '미기재', 'N/A'}

# 문자열 표준화: 공백 정리, 구분자 ' > '로 통일
s = df['주제분야'].astype(str).str.strip()

# '인문학  >  역사학' 같은 경우 공백 정리 및 다양한 기호 통일
s = s.replace(r'\s*>\s*', ' > ', regex=True)   # 구분자 좌우 공백 통일
s = s.replace(r'\s+', ' ', regex=True)         # 다중 공백 → 단일 공백

# 유사결측을 NA로 치환
s_lower = s.str.lower()
s = s.where(~s_lower.isin({x.lower() for x in NULL_LIKE}), pd.NA)

# 결측 채움(분석 편의용 라벨)
s = s.fillna('미상')

# 계층 분해: 대/중/소 분류 (최대 3단계로 가정함, 더 깊으면 마지막에 합침) 
# 단계가 많은 예시 : 인문학 > 한국어와문학 > 국어학 > 국어사  
parts = s.str.split(' > ', n=2, expand=True)  # n=2 ⇒ 최대 3조각
parts.columns = ['대분류','중분류','소분류']

# 소분류에 여전히 ' > '가 남았다면(4단계 이상) 마지막 조각에 이어붙여진 상태이므로 그대로 둡니다.

# 정리된 컬럼 반영
df['주제분야_표준'] = s
df[['대분류','중분류','소분류']] = parts

# 타입: 범주형으로 변환(메모리 절약 + 그룹화 속도)
for c in ['주제분야_표준','대분류','중분류','소분류']:
    df[c] = df[c].astype('category')


In [ ]:
# 연도별 데이터 개수
print(df['발행년도'].value_counts().sort_index())

# 연도별 중분류 개수
print(df.groupby('발행년도')['중분류'].nunique())

# 중분류 목록
print(df['중분류'].unique())


### 데이터 저장


In [ ]:
from pathlib import Path

# 0) 결과 폴더 생성: SS/results_SS
out_dir = Path.cwd() / "results_SS"
out_dir.mkdir(parents=True, exist_ok=True)

# 1) 저장 대상 DF (df 사용)
df_to_save = df.reset_index(drop=True)

# 2) 파일 경로 (파일명 앞에 SS 사회과학 접두사 붙임)
csv_path = out_dir / "SS_eng_abstract_clean.csv"
pkl_path = out_dir / "SS_eng_abstract_clean.pkl"

# 3) 저장
df_to_save.to_csv(csv_path, index=False, encoding="utf-8-sig")
df_to_save.to_pickle(pkl_path)

# 4) 로그 & 간단 점검
print("[저장 완료]")
print("Rows:", len(df_to_save))
print("CSV   →", csv_path.resolve())
print("Pickle →", pkl_path.resolve())
if "발행년도" in df_to_save.columns:
    print("Year range:", int(df_to_save["발행년도"].min()), "→", int(df_to_save["발행년도"].max()))
if "영어초록" in df_to_save.columns:
    print("영어초록 NaN 개수:", int(df_to_save["영어초록"].isna().sum()))

### csv 미리 보기

In [ ]:
from pathlib import Path
import pandas as pd

csv_path = Path.cwd() / "results_SS" / "SS_eng_abstract_clean.csv"

# 1) 존재 확인
if not csv_path.exists():
    raise FileNotFoundError(f"파일이 없습니다: {csv_path}")

# 2) 로드 (저장 시 UTF-8-SIG)
df_loaded = pd.read_csv(csv_path, encoding="utf-8-sig", low_memory=False)

# 3) 간단 미리보기
print("[로드 완료]")
print("shape:", df_loaded.shape)
print("columns:", list(df_loaded.columns))
display(df_loaded.head(5)) 


# 2) 토크나이징 및 단어 행렬 생성

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.feature_extraction.text import CountVectorizer
import joblib

# ------------------------------
# 0) 경로 (SS)
# ------------------------------
in_pkl  = Path("results_SS/SS_eng_abstract_clean.pkl")
out_dir = Path("results_SS")
out_dir.mkdir(parents=True, exist_ok=True)

# 산출물 경로 정의 (SS용)
out_word_year_counts = out_dir / "SS_word_year_matrix_counts.csv"  # [행=단어, 열=연도] 각 연도에서 '그 단어가 등장한 문서 수' 행렬
out_doc_counts       = out_dir / "SS_doc_counts.csv"               # 연도별 전체 문서 수(n_docs); 분모로 사용
out_vectorizer_pkl   = out_dir / "SS_vectorizer.pkl"               # CountVectorizer 객체 향후 재현/재사용에 필수
out_terms_txt        = out_dir / "SS_terms.txt"                    # vectorizer의 feature_names 순서 그대로의 단어 리스트(행 정렬 키)
out_years_npy        = out_dir / "SS_years.npy"                    # counts 열(연도) 순서 배열(2004..2024). 열 정렬 키


# ------------------------------
# 1) 데이터 로드 (신뢰 + 얇은 검증)
# ------------------------------
if not in_pkl.exists():
    raise FileNotFoundError(f"입력 없음: {in_pkl}")

df = pd.read_pickle(in_pkl)
df = df.rename(columns={"영어초록": "abstract_en"})[["발행년도", "abstract_en"]]

# 얇은 검증(빠르고 안전)
assert df["발행년도"].between(2004, 2024).all(), "예상 밖 연도 발견"
assert df["abstract_en"].notna().all(), "영어초록 NaN 발견"

# 연도 정렬용
df["발행년도"] = df["발행년도"].astype(int)
years = np.sort(df["발행년도"].unique())
print(f"[로드 완료] {len(df)}개 문서, {len(years)}개 연도({years[0]}..{years[-1]})")


# ------------------------------
# 2) 벡터화 (binary 존재 여부 기준)
# ------------------------------
vectorizer = CountVectorizer(
    binary=True,
    min_df=0.0001,
    lowercase=True,
    token_pattern=r"(?u)\b[A-Za-z][A-Za-z-]{1,}\b"
)
X = vectorizer.fit_transform(df["abstract_en"])
terms = vectorizer.get_feature_names_out()

# ------------------------------
# 3) 연도별 counts 집계
# ------------------------------
doc_counts = df.groupby("발행년도").size().reindex(years, fill_value=0)
doc_counts.to_csv(out_doc_counts, header=["n_docs"])

year_row_indices = {y: np.flatnonzero(df["발행년도"].values == y) for y in years}
counts_mat = np.zeros((len(terms), len(years)), dtype=np.int32)

for j, y in enumerate(years):
    rows = year_row_indices[y]
    if rows.size == 0:
        continue
    X_y = X[rows, :]
    counts = (X_y > 0).sum(axis=0).A1
    counts_mat[:, j] = counts

word_year_counts = pd.DataFrame(counts_mat, index=terms, columns=years)
word_year_counts.to_csv(out_word_year_counts, encoding="utf-8-sig")

# ------------------------------
# 4) 재현용 산출물 (vectorizer/terms/years) 생성
# ------------------------------
joblib.dump(vectorizer, out_vectorizer_pkl)
np.save(out_years_npy, years)
with open(out_terms_txt, "w", encoding="utf-8") as f:
    f.writelines(t + "\n" for t in terms)

# ------------------------------
# 5) 로그
# ------------------------------
print("✅ 완료:")
print(" - 연도별 문서 수:", out_doc_counts)
print(" - 단어-연도 카운트 행렬:", out_word_year_counts)
print(" - Vectorizer:", out_vectorizer_pkl)
print(" - Terms list:", out_terms_txt)
print(" - Years.npy:", out_years_npy)
print(" - terms 수:", len(terms), "/ 문서 수:", X.shape[0])


결과 : 511,787편의 사회과학 영어 초록 중 23,314개 유효 단어가 잡힘


# 3) 연도별 빈도 계산 및 2024년 과잉 어휘 식별

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

# ------------------------------
# 0) 경로
# ------------------------------

base_dir = Path("results_SS")  
counts_path = base_dir / "SS_word_year_matrix_counts.csv"
docs_path   = base_dir / "SS_doc_counts.csv"

out_full    = base_dir / "SS_excess_words_2024.csv"          # 2024년 분석 대상 모든 단어의 결과
out_filtered= base_dir / "SS_excess_words_2024_filtered.csv" # 임계값 필터하여 과잉어휘만 모은 결과


# ------------------------------
# 1) 데이터 로드
# ------------------------------
counts = pd.read_csv(counts_path, index_col=0)   # 행 = terms, 열 = years
doc_counts = pd.read_csv(docs_path, index_col=0)["n_docs"]  # 인덱스 = years, 밸류 = n_docs

# 분석에 쓸 연도들
YEARS = [2021, 2022, 2023, 2024]
missing = [y for y in YEARS if str(y) not in counts.columns or y not in doc_counts.index]
if missing:
    raise ValueError(f"필요 연도 누락: {missing}")

# ------------------------------
# 2) 단어의 연도별 등장비율 p 계산,  p = (a+1)/(b+1) 
# ------------------------------
p_s = {}
for y in YEARS:
    a = counts[str(y)].astype(float)
    b = float(doc_counts.loc[y])
    p_s[y] = (a + 1.0) / (b + 1.0) #a는 그 연도에 해당 단어가 등장한 문서 수, b는 그 연도 전체 문서 수
    # 즉, 그 연도 전체 문서 중 몇 %에서 해당 단어가 쓰였는가. +1은 0으로 나누는 걸 방지함

p_df = pd.DataFrame(p_s)  # p_df는 index = 단어, columns = [2021, 2022, 2023, 2024] 형태의 “연도별 등장비율 행렬”
p_df.index.name = "term" # 각 단어(term)가 2021–2024년 각 연도 초록 중 몇 %에서 등장했는가를 담은 비율 행렬

# ------------------------------
# 3) 분석 대상 단어 선택 (빈도 ≥ 1e-4 in 2024 AND 2023) 
# 빈도 ≥ 1e-4는 희귀한 잡음을 제거하려는 목적이므로 단어선정의 안정성을 위해 2023년도도 포함함
# (아래의 예상치 계산에서는 2023년도는 사용하지 않는다는 점이 다름)
# ------------------------------
min_p = 1e-4
mask_keep = (p_df[2024] >= min_p) & (p_df[2023] >= min_p)
p_df = p_df.loc[mask_keep].copy()

# ------------------------------
# 4) q(2024 예상치) 계산 - 외삽 규칙
#     q = p2022 + 2*max(p2022 - p2021, 0)   # (증가 추세면 2배 증가, 감소 추세면 현상 유지)
# -> 하락한 단어는 증가하지 않는다는 보수적 가정, 즉, q >= p2022
# -> 상승한 단어는 증가 추세를 반영해서 2배 반영, q를 높게 잡아 δ = p − q가 부풀려지지 않도록(보수적 하한) 설계
# ------------------------------
p21 = p_df[2021].values
p22 = p_df[2022].values
p24 = p_df[2024].values

trend = np.maximum(p22 - p21, 0.0)
q = p22 + 2.0 * trend
q = np.maximum(q, p22)  # 보수적 하한

# ------------------------------
# 5) δ, ratio, log10 r 계산
# ------------------------------
eps = 1e-12
delta = p24 - q
ratio = p24 / np.maximum(q, eps)
log10r = np.log10(np.maximum(ratio, eps))

# ------------------------------
# 6) 결과 테이블 구성
# ------------------------------
out = pd.DataFrame({
    "p2021": p_df[2021],
    "p2022": p_df[2022],
    "p2023": p_df[2023],
    "p2024": p_df[2024],
    "q2024": q,
    "delta": delta,
    "ratio": ratio,
    "log10_ratio": log10r,
})

# 정렬: δ와 ratio 모두 큰 순으로 참고하기 좋도록
out_sorted = out.sort_values(["delta", "ratio"], ascending=[False, False])

# 전체 저장
out_sorted.to_csv(out_full, encoding="utf-8-sig")

# ------------------------------
# 7) 임계값 필터 (수정 버전)
#    - δ > 0.01  또는  log10 r > 0.1 (고정 컷)
#    - + 빈도 필터: p2024, p2023 ≥ 1e-4
# ------------------------------
gap_thr = 0.01          # 절대적 빈도 차이 기준; 0.01 = 1%p 이상의 과잉만 채택
logr_thr = 0.1          #  log10(r) 고정 컷; 너무 많은 단어가 통과하지 않도록 보수적으로 설정
min_freq = 1e-4         # 분석 연도(Y)와 직전 연도(Y-1) 모두 p ≥ 1e-4인 단어만 대상
eps = 1e-12             # 수치 안정성용 아주 작은 값(0 나눗셈, log(0) 방지)

# 빈도 필터 선적용 (Y와 Y-1에서 p ≥ 1e-4)
# 극저빈도(희귀) 단어는 작은 변동에도 δ, r이 크게 출렁이며 log 기준을 쉽게 통과하므로
# 먼저 제거해 과검출을 줄임
mask_freq = (out_sorted["p2024"] >= min_freq) & (out_sorted["p2023"] >= min_freq)
cand = out_sorted[mask_freq]

# --- 임계식 적용 ---
# 원논문과 다른 점은 log 기준을 고정값(0.1)로 보수화하여 과검출을 억제함.
#  원 논문의 log 컷은 대략 0.075(≈19% 증가) 이며 이 분석의 경우 0.1(≈26% 증가)로 상향 조정해서 과검출(거짓양성)을 줄임.
filtered = cand[
    (cand["delta"] > gap_thr) |
    (cand["log10_ratio"] > logr_thr)
]

# 임계값 통과 단어 수 (필터 전/후 모두 표기)
print(f"임계값 통과 단어 수: {len(filtered)} / {len(cand)} (원본 {len(out_sorted)})")

# 저장
filtered.to_csv(out_filtered, encoding="utf-8-sig")

print("✅ 완료:")
print(" - 전체 표  :", out_full)
print(" - 임계값 표:", out_filtered)  # 최종 Excess words 리스트
print(" - 대상 단어 수(원본):", len(out_sorted))
print(" - 빈도 필터 후 대상:", len(cand))        
print(" - 임계 통과 수     :", len(filtered))



# 4) 과잉 어휘 분류

In [ ]:
# delta 및 ratio 기준 상위 과잉어휘 100 + 100 병합 --> 수동 라벨링용 파일 생성

import pandas as pd
from pathlib import Path  

base = Path("results_SS")
filtered = pd.read_csv(base / "SS_excess_words_2024_filtered.csv")

# 인덱스 복원/표준화: 'term' → 'word' 로 통일
filtered = filtered.reset_index(drop=True)
if "word" not in filtered.columns:
    if "term" in filtered.columns:
        filtered = filtered.rename(columns={"term": "word"})
    elif "index" in filtered.columns:
        filtered = filtered.rename(columns={"index": "word"})
    elif "Unnamed: 0" in filtered.columns:
        filtered = filtered.rename(columns={"Unnamed: 0": "word"})
    else:
        filtered = filtered.reset_index().rename(columns={"index": "word"})

# 숫자 컬럼 보정 (안전)
for col in ("delta", "ratio"):
    filtered[col] = pd.to_numeric(filtered[col], errors="coerce")

# 전체 데이터 기준 랭킹 계산(참고용)
tmp = filtered.copy()
tmp["delta_rank"] = tmp["delta"].rank(ascending=False, method="min")
tmp["ratio_rank"] = tmp["ratio"].rank(ascending=False, method="min")

# Top100 추출
top_delta = filtered.nlargest(100, "delta")
top_ratio = filtered.nlargest(100, "ratio")

# 합집합 + 중복 제거
combined = pd.concat([top_delta, top_ratio], ignore_index=True).drop_duplicates("word")

# 랭크 열 병합
combined = combined.merge(tmp[["word", "delta_rank", "ratio_rank"]], on="word", how="left")

# 라벨 컬럼 없으면 생성
if "manual_label" not in combined.columns:
    combined["manual_label"] = ""   # 이후 수동 라벨링: style/content 등

# === 출력 파일명: ===
out_csv   = base / "SS_excess_words_top_combined_for_labeling.csv"
out_excel = base / "SS_excess_words_top_combined_for_labeling.xlsx"

combined.to_csv(out_csv, index=False, encoding="utf-8-sig")
combined.to_excel(out_excel, index=False)

print(f"✅ 총 {len(combined)}개 단어가 추출되어 저장되었습니다. → {out_csv}")
print(f"✅ 엑셀 파일로도 저장되었습니다. → {out_excel}")


# 5-1) p vs ratio

x축은 2024년 실제 빈도(p), y축은 예상 대비 과잉 비율(r = p/q)임.

In [ ]:
# Figure 2A (SS): 2024 빈도(p) vs 과잉 비율(r=p/q), 수동 라벨(manual_label)로 색상 구분
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pathlib import Path

# ------------------------------
# 1) 경로 & 데이터 로드 (SS)
# ------------------------------
base = Path("results_SS")
in_xlsx = base / "SS_excess_words_top_labeled.xlsx" #라벨링한 파일
df = pd.read_excel(in_xlsx)

# ------------------------------
# 2) 열 이름 고정
# ------------------------------
COL_P = "p2024"
COL_RATIO = "ratio"
COL_LABEL = "manual_label"
COL_WORD = "word"

# ------------------------------
# 3) 전처리 
# ------------------------------
# p>0, ratio>1.5 구간만 (원 코드 기준)
df = df[(df[COL_P] > 0) & (df[COL_RATIO] > 1.5)].copy()

# 라벨 누락/대소문자/공백 정리
df[COL_LABEL] = (
    df[COL_LABEL]
    .astype(str)
    .str.strip()
    .str.lower()
    .replace({"": "content", "nan": "content"})
)

# ------------------------------
# 4) 시각화
# ------------------------------
palette = {"style": "orange", "content": "blue", "ambiguous": "gray"}
present_labels = [lab for lab in ["style","content","ambiguous"] if lab in df[COL_LABEL].unique()]

sns.set(style="whitegrid", font_scale=1.1)
plt.figure(figsize=(10, 8))
sns.scatterplot(
    data=df, x=COL_P, y=COL_RATIO,
    hue=COL_LABEL, palette=palette, hue_order=present_labels or None,
    s=50, edgecolor="k", alpha=0.8
)

# 로그축
plt.xscale("log")
plt.yscale("log")

# y축 하한(=1.5) 고정, 상한은 데이터에 맞게 자동
plt.ylim(1.5, None)

# 임계선: ratio = p**4 (가이드라인)
p_min = max(df[COL_P].min(), 1e-8)
p_max = df[COL_P].max()
p_grid = np.logspace(np.log10(p_min), np.log10(p_max), 200)
plt.plot(p_grid, p_grid**4, linestyle="dashed", color="black", linewidth=1)

# 라벨: ratio 상위 15개
top_words = df.sort_values(by=COL_RATIO, ascending=False).head(15)
for _, r in top_words.iterrows():
    plt.text(r[COL_P], r[COL_RATIO], str(r[COL_WORD]), fontsize=9)

plt.xlabel("p2024 (Frequency in 2024)")
plt.ylabel("Ratio (p / q)")
plt.title("SS Figure 2A: 2024 frequency vs ratio (ratio > 1.5)")
plt.legend(title="Label", loc="upper left")

# ------------------------------
# 5) 저장
# ------------------------------
fig_dir = base / "figures"
fig_dir.mkdir(parents=True, exist_ok=True)
plt.tight_layout()
plt.savefig(fig_dir / "SS_figure2A_p_vs_ratio.png", dpi=300)
plt.savefig(fig_dir / "SS_figure2A_p_vs_ratio.pdf")
plt.show()


# 5-2) p vs delta
x축은 2024년 단어 빈도(p), y축은 과잉 정도(Δ)를 나타냄.

Δ > 0.005 이상인 단어만 필터링

In [ ]:
# Figure 2B (SS): 2024 빈도(p) vs delta(p - q)
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import numpy as np
from pathlib import Path

# 0) adjustText 폴백
try:
    from adjustText import adjust_text
    HAS_ADJUST = True
except Exception:
    HAS_ADJUST = False

# 1) 데이터 로드 (SS 라벨링 파일)
base = Path("results_SS")
in_xlsx = base / "SS_excess_words_top_labeled.xlsx"   # 라벨링한 파일
df = pd.read_excel(in_xlsx)

# 2) 컬럼 이름 매핑
def pick(d, candidates, required=True):
    for c in candidates:
        if c in d.columns:
            return c
    if required:
        raise KeyError(f"Required column not found. Tried: {candidates}. Available: {list(d.columns)}")
    return None

COL_P      = pick(df, ["p2024", "p", "p_2024"])
COL_Q      = pick(df, ["q2024", "q", "q_2024"])
COL_DELTA  = "delta"
COL_WORD   = pick(df, ["word", "term"], required=False) or "word"
COL_LABEL  = pick(df, ["manual_label", "label", "type"], required=False) or "manual_label"

df = df.copy()

# 3) delta 없으면 생성
if COL_DELTA not in df.columns:
    df[COL_DELTA] = df[COL_P] - df[COL_Q]

# 4) 전처리: p>0, delta>0.005, 라벨 정규화
df[COL_LABEL] = df.get(COL_LABEL, "content")
df[COL_LABEL] = df[COL_LABEL].fillna("content").astype(str).str.strip().str.lower()

filtered_df = df[(df[COL_P] > 0) & (df[COL_DELTA] > 0.005)].copy()

# 5) 색상 매핑
color_map = {"style": "orange", "content": "blue", "ambiguous": "gray"}
filtered_df["__color__"] = filtered_df[COL_LABEL].map(color_map).fillna("gray")

# 6) 시각화
plt.figure(figsize=(10, 7))
plt.scatter(
    filtered_df[COL_P],
    filtered_df[COL_DELTA],
    c=filtered_df["__color__"],
    alpha=0.7,
    edgecolors="k",
    linewidths=0.5,
)

plt.xlabel(f"{COL_P} (Frequency in 2024)")
plt.ylabel("Delta (p - q)")
plt.title("SS Figure 2B: 2024 frequency vs delta (delta > 0.005)")
plt.grid(True, linestyle="--", alpha=0.5)
plt.xscale("log")

# 7) 텍스트 라벨링: delta 상위 30개
top_labels = filtered_df.nlargest(30, COL_DELTA)
texts = []
for _, row in top_labels.iterrows():
    texts.append(plt.text(row[COL_P], row[COL_DELTA], str(row.get(COL_WORD, "")), fontsize=9))

if HAS_ADJUST and texts:
    adjust_text(
        texts,
        arrowprops=dict(arrowstyle="-", color="black", lw=0.5),
        only_move={"points": "y", "text": "y"},
        expand_text=(1.05, 1.2),
    )

# 8) 범례
legend_elements = [
    Patch(facecolor="orange", edgecolor="k", label="Style"),
    Patch(facecolor="blue", edgecolor="k", label="Content"),
    Patch(facecolor="gray", edgecolor="k", label="Ambiguous"),
]
plt.legend(handles=legend_elements, title="Label", loc="upper left")

# 9) 저장
fig_dir = base / "figures"
fig_dir.mkdir(parents=True, exist_ok=True)
plt.tight_layout()
plt.savefig(fig_dir / "SS_figure2B_p_vs_delta.png", dpi=300)
plt.savefig(fig_dir / "SS_figure2B_p_vs_delta.pdf")
plt.show()


# 5-3) 9개 단어에 대한 연도별 시각화

In [ ]:
# SS_figure1A_fixed_list.py
# Figure 1A (Social Sciences): 고정 9단어 + 연도별 p_t와 2024 예상선(q_2024)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# -----------------------------
# 0) 경로 (SS)
# -----------------------------
labels_path      = Path("results_SS/SS_excess_words_top_labeled.xlsx")
word_counts_path = Path("results_SS/SS_word_year_matrix_counts.csv")
doc_counts_path  = Path("results_SS/SS_doc_counts.csv")

# -----------------------------
# 1) 데이터 로드
# -----------------------------
labels = pd.read_excel(labels_path)
labels.columns = [str(c).strip() for c in labels.columns]

# 단어-연도 '문서수 기준' 카운트 행렬
if not word_counts_path.exists():
    raise FileNotFoundError(f"단어-연도 카운트 행렬 CSV 없음: {word_counts_path}")
word_matrix = pd.read_csv(word_counts_path, index_col=0)

# 연도별 문서 수 (SS에서는 n_docs 단일 컬럼 + 인덱스=연도 형태)
if not doc_counts_path.exists():
    raise FileNotFoundError(f"연도별 문서 수 CSV 없음: {doc_counts_path}")
doc_counts = pd.read_csv(doc_counts_path, index_col=0).iloc[:, 0]  # Series (index=year, name=n_docs)
doc_counts.index = doc_counts.index.map(lambda x: str(x).strip())

# -----------------------------
# 2) 라벨·단어 컬럼 정규화 + (고정 리스트 사용)
# -----------------------------
def pick(df, candidates, required=True):
    cols = [c for c in candidates if c in df.columns]
    if cols: return cols[0]
    if required:
        raise KeyError(f"Required column not found. Tried: {candidates}. Available: {list(df.columns)}")
    return None

COL_WORD  = pick(labels, ["word", "term"])
COL_LABEL = pick(labels, ["manual_label", "label", "type"])

lab_df = labels[[COL_WORD, COL_LABEL]].copy()
lab_df[COL_WORD]  = lab_df[COL_WORD].astype(str).str.lower().str.strip()
lab_df[COL_LABEL] = lab_df[COL_LABEL].astype(str).str.lower().str.strip()

# 🔒 고정 9단어 (인문학은 "showcasing", "underscoring", "aligning", "additionally", "intricate", "notably", "delving", "reshaping", "gpt")
target_words_raw = [
    "additionally", "underscoring", "aligning",
    "explores", "notably" , "comprehensive",
    "ai", "delving", "these"
]

# word_matrix 인덱스 정리 및 교차 확인
word_matrix.index = word_matrix.index.astype(str).str.lower().str.strip()
present = [w for w in target_words_raw if w in word_matrix.index]
missing = [w for w in target_words_raw if w not in word_matrix.index]

print("[대상 단어] (존재):", present)
if missing:
    print("[경고] 단어-연도 행렬에 없는 단어:", missing)

target_words = present  # 없는 단어 제외

# 라벨 맵(색상용)
label_map = dict(zip(lab_df[COL_WORD], lab_df[COL_LABEL]))
color_by_label = {"style": "tab:orange", "content": "tab:blue", "ambiguous": "gray"}

# -----------------------------
# 3) 연도 교집합 및 p 계산/외삽
# -----------------------------
years_all = [str(y) for y in range(2004, 2025)]  # SS: 2004..2024
word_matrix.columns = [str(c).strip() for c in word_matrix.columns]
years = [y for y in years_all if (y in word_matrix.columns) and (y in doc_counts.index)]
if not years:
    raise ValueError("공통 연도가 없습니다. word_matrix 컬럼/문서수 인덱스를 확인하세요.")

def compute_freq(word: str):
    """연도별 p=(a+1)/(b+1). a:해당 단어 포함 문서 수, b:해당 연도 전체 문서 수"""
    a = word_matrix.loc[word, years].astype(float) if word in word_matrix.index else pd.Series(0.0, index=years)
    b = doc_counts.loc[years].astype(float)  # n_docs Series
    p = (a + 1.0) / (b + 1.0)
    return p.values.tolist()

def extrapolate_q(p2021, p2022):
    """Kobak식: q_2024 = p2022 + 2 * max(p2022 - p2021, 0)"""
    return p2022 + 2 * max(p2022 - p2021, 0)

# -----------------------------
# 4) 시각화
# -----------------------------
n = len(target_words)
rows, cols = 3, 3
fig, axes = plt.subplots(rows, cols, figsize=(15, 12))
axes = axes.flatten()

for i, word in enumerate(target_words):
    ax = axes[i]
    p_t = compute_freq(word)
    lab = label_map.get(word, "content")
    color = color_by_label.get(lab, "gray")

    ax.plot(years, p_t, label=f"{word}", color=color, linewidth=2)

    if ("2021" in years) and ("2022" in years) and ("2024" in years):
        p2021 = p_t[years.index("2021")]
        p2022 = p_t[years.index("2022")]
        q_2024 = extrapolate_q(p2021, p2022)
        ax.plot(["2022", "2024"], [p2022, q_2024],
                color="black", linestyle="--", linewidth=1.2, label="Expected q (2024)")

    ax.set_title(f"{word}  [{lab}]", fontsize=11)
    ax.tick_params(axis='x', labelrotation=45)
    ax.set_ylim(bottom=0)
    ymin, ymax = ax.get_ylim()
    ax.set_ylim(0, max(ymax, (max(p_t) if p_t else 0) * 1.15 + 1e-6))

# 남는 서브플롯 제거
for j in range(i + 1, rows * cols):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.suptitle("SS Figure 1A: Word frequency over time with 2024 projection",
             fontsize=16, y=1.02)
out_dir = Path("results_SS/figures"); out_dir.mkdir(parents=True, exist_ok=True)
plt.savefig(out_dir / "SS_figure1A_word_timecourse_fixed.png", dpi=300, bbox_inches="tight")
plt.savefig(out_dir / "SS_figure1A_word_timecourse_fixed.pdf", bbox_inches="tight")
plt.show()
